In [1]:
%run Preprocessing_main.ipynb

In [2]:
MVP_data['G_x'] = MVP_data['G_x'].astype(int)

In [3]:
def drop_col(data):
    corr = data.corr()
    NEW_data = data
    del_list = []
    for i in corr:
        for ind in corr[(corr[i]>0.9)&(corr[i]<1)].index:
            if ind not in del_list:
                try:
                    NEW_data = NEW_data.drop(ind, axis = 1)
                except:
                    continue
        del_list.append(i)
    NEW_data = NEW_data.drop('Player', axis = 1)
    return NEW_data

In [4]:
NEW_data = drop_col(MVP_data)

In [5]:
# standardize through years
def std_scale_split(data, year_names):
    scaler = StandardScaler()
    X_train_result = pd.DataFrame()
    X_test_result = pd.DataFrame()
    y_train_result = pd.DataFrame()
    y_test_result = pd.DataFrame()
    for i in year_names:
        
        data_year = data[data['year_y']==i]
        data_year = data_year.drop(columns='year_y')
        
        X = data_year.iloc[:, :-1]
        y = data_year.iloc[:, -1]

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 1)
        train_scaled = scaler.fit_transform(X_train)
        test_scaled = scaler.transform(X_test)
        
        X_train_result = pd.concat([X_train_result, pd.DataFrame(train_scaled)])
        X_test_result = pd.concat([X_test_result, pd.DataFrame(test_scaled)])
        y_train_result = pd.concat([y_train_result, y_train])
        y_test_result = pd.concat([y_test_result, y_test])
        
    return X_train_result, X_test_result, y_train_result, y_test_result

X_train, X_test, y_train, y_test = std_scale_split(NEW_data, year_names)

In [6]:
regressor = LinearRegression()
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)
y_train_pred = regressor.predict(X_train)
print(regressionSummary(y_train, y_train_pred))
print(regressionSummary(y_test, y_pred))
lin_mse = mean_squared_error(y_test, y_pred)


Regression statistics

                      Mean Error (ME) : 0.0000
       Root Mean Squared Error (RMSE) : 0.1983
            Mean Absolute Error (MAE) : 0.1541
          Mean Percentage Error (MPE) : -480.8082
Mean Absolute Percentage Error (MAPE) : 3054.9236
None

Regression statistics

                      Mean Error (ME) : -0.0171
       Root Mean Squared Error (RMSE) : 0.1603
            Mean Absolute Error (MAE) : 0.1254
          Mean Percentage Error (MPE) : -192.2601
Mean Absolute Percentage Error (MAPE) : 922.2176
None


In [7]:
lin_r2 = r2_score(np.array(y_test).reshape(len(y_test)), np.array(y_pred).reshape(len(y_pred)))

In [8]:
regressor = SVR()
regressor.fit(X = X_train, y = y_train)
y_pred = regressor.predict(X_test)
y_train_pred = regressor.predict(X_train)
print(regressionSummary(y_train, y_train_pred))
print(regressionSummary(y_test, y_pred))
SVR_mse = mean_squared_error(y_test, y_pred)


Regression statistics

                      Mean Error (ME) : -0.0128
       Root Mean Squared Error (RMSE) : 0.1078
            Mean Absolute Error (MAE) : 0.0824
          Mean Percentage Error (MPE) : -1233.9688
Mean Absolute Percentage Error (MAPE) : 1890.8296
None

Regression statistics

                      Mean Error (ME) : -0.0461
       Root Mean Squared Error (RMSE) : 0.2011
            Mean Absolute Error (MAE) : 0.1626
          Mean Percentage Error (MPE) : -1089.9168
Mean Absolute Percentage Error (MAPE) : 1289.5220
None


In [9]:
SVR_r2 = r2_score(np.array(y_test).reshape(len(y_test)), np.array(y_pred).reshape(len(y_pred)))

In [10]:
fin_alpha = 0
fin_r2 = -1E20
for alpha in np.linspace(0, 0.5, 21):
    regressor = DecisionTreeRegressor(ccp_alpha = alpha, random_state = 1)
    regressor.fit(X = X_train, y = np.array(y_train).reshape(len(y_train), ))
    y_pred = regressor.predict(X_test)
    y_train_pred = regressor.predict(X_train)
    r2 = r2_score(np.array(y_test).reshape(len(y_test)), np.array(y_pred).reshape(len(y_pred)))
    if r2 > fin_r2:
        fin_alpha = alpha
        fin_r2 = r2
print('alpha: {:1.4f}, R2: {:1.4f}'.format(fin_alpha, fin_r2))
tree_r2 = fin_r2

alpha: 0.0500, R2: -0.0185


In [11]:
min_mse = 1E20
fin_alpha = 0
fin_NE = 0
fin_r2 = -1E20

for alpha in np.linspace(0, 0.5, 11):
    for n_e  in np.linspace(10, 100, 10):
        regressor = RandomForestRegressor(n_estimators = int(n_e), ccp_alpha = alpha, random_state = 1)
        regressor.fit(X = X_train, y = np.array(y_train).reshape(len(y_train), ))
        y_pred = regressor.predict(X_test)
        y_train_pred = regressor.predict(X_train)
        MSE = mean_squared_error(y_test, y_pred)
        r2 = r2_score(np.array(y_test).reshape(len(y_test)), np.array(y_pred).reshape(len(y_pred)))
        if r2 > fin_r2:
            fin_alpha = alpha
            fin_r2 = r2
print('alpha: {:1.4f}, R2: {:1.4f}'.format(fin_alpha, fin_r2))
for_r2 = fin_r2

alpha: 0.0500, R2: 0.1553


### Neural Network

In [12]:
# standardize through years
def std_scale_split(data, year_names):
    scaler = StandardScaler()
    X_train_result = pd.DataFrame()
    X_test_result = pd.DataFrame()
    y_train_result = pd.DataFrame()
    y_test_result = pd.DataFrame()
    for i in year_names:
        
        data_year = data[data['year_y']==i]
        data_year = data_year.drop(columns='year_y')
        
        X = data_year.iloc[:, :-1]
        y = data_year.iloc[:, -1]
        
        scaler = MinMaxScaler()
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 1)
        train_scaled = scaler.fit_transform(X_train)
        test_scaled = scaler.transform(X_test)
        
        X_train_result = pd.concat([X_train_result, pd.DataFrame(train_scaled)])
        X_test_result = pd.concat([X_test_result, pd.DataFrame(test_scaled)])
        y_train_result = pd.concat([y_train_result, y_train])
        y_test_result = pd.concat([y_test_result, y_test])
        
    return X_train_result, X_test_result, y_train_result, y_test_result

X_train, X_test, y_train, y_test = std_scale_split(NEW_data, year_names)

In [13]:
best_r2 = -1E20
best_bat = 0
best_size = 0

for size1 in [int(i) for i in np.linspace(5, 20, 16)]:
    for bat in [int(i) for i in np.linspace(0, 100, 21)]:
        NNRegressor = MLPRegressor(hidden_layer_sizes = (size1), batch_size = bat, 
                                   activation = 'logistic', random_state= 1)
        NNRegressor.fit(X_train, y_train)
        y_pred = NNRegressor.predict(X_test)
        NN_r2 = r2_score(np.array(y_test).reshape(len(y_test)), np.array(y_pred).reshape(len(y_pred)))
        if NN_r2 > best_r2:
            best_r2 = NN_r2
            best_size = size1
            best_bat = bat
print('layer size: {:1.4f}, batch size: {:1.4f}, R2: {:1.4f}'.format(best_size, best_bat, best_r2))
NN_r2 = best_r2

layer size: 8.0000, batch size: 45.0000, R2: 0.4101


In [14]:
print('Linear regression: {:1.4f}'.format(lin_r2))
print('SVR: {:1.4f}'.format(SVR_r2))
print('Decision tree: {:1.4f}'.format(tree_r2))
print('Random forest: {:1.4f}'.format(for_r2))
print('Neural Network: {:1.4f}'.format(NN_r2))

Linear regression: 0.1980
SVR: -0.2610
Decision tree: -0.0185
Random forest: 0.1553
Neural Network: 0.4101


In [15]:
NNRegressor = MLPRegressor(hidden_layer_sizes = (16), batch_size = 45, 
                                   activation = 'logistic', random_state= 1)
NNRegressor.fit(X_train, y_train)
y_pred = NNRegressor.predict(X_test)
r2_best_corr = r2_score(np.array(y_test).reshape(len(y_test)), np.array(y_pred).reshape(len(y_pred)))
print('layer size: {:1.4f}, batch size: {:1.4f}, R2: {:1.4f}'.format(16, 45, r2_best_corr))

layer size: 16.0000, batch size: 45.0000, R2: 0.2677
